In [2]:
import pandas as pd
import numpy as np
import datetime
import json
import cv2
import matplotlib.pyplot as plt

# metric calculating libraries

from new_metric import main_metric, check_true

In [17]:
gts_folder = "Groundtruth_Markup//"
preds_folder = "Predictions//"

video_gt = {}
with open(gts_folder + "VIDEO_val_groundtruth.json", "r") as read_file:
    video_gt = json.load(read_file)
    
FLIR_val_gt = {}
with open(gts_folder + "FLIR_val_groundtruth.json", "r") as read_file:
    FLIR_val_gt = json.load(read_file)
    
############
     
video_preds = {}
with open(preds_folder + "VIDEO_val_predictions_TSMO_exp.json", "r") as read_file:
    video_preds = json.load(read_file)
    
FLIR_val_preds = {}
with open(preds_folder + "FLIR_val_predictions_TSMO_exp.json", "r") as read_file:
    FLIR_val_preds = json.load(read_file)

In [41]:
# gts_folder = "Groundtruth_Markup//"
# preds_folder = "Predictions//"

# video_gt = {}
# with open(gts_folder + "VIDEO_train_groundtruth.json", "r") as read_file:
#     video_gt = json.load(read_file)
    
# FLIR_val_gt = {}
# with open(gts_folder + "FLIR_train_groundtruth.json", "r") as read_file:
#     FLIR_val_gt = json.load(read_file)
    
# ############
     
# video_preds = {}
# with open(preds_folder + "VIDEO_train_predictions_TSMO.json", "r") as read_file:
#     video_preds = json.load(read_file)
    
# FLIR_val_preds = {}
# with open(preds_folder + "FLIR_train_predictions_TSMO.json", "r") as read_file:
#     FLIR_val_preds = json.load(read_file)

In [13]:
metric_type = 'modified' # standart or modified

In [43]:
%%time
columns = ['Frame_Number', 'FLIR0_Video1', 'Object_Index', 'Start_Time', 'Detected', 'Max_Intersected_CNT', 'IOU', 
          'X', 'Y', 'W', 'H', 'Frame_Int', 'GT_Int']
video_gt_table = pd.DataFrame(columns=columns)
video_gt_table = video_gt_table.astype({"Frame_Number": str, 'FLIR0_Video1': np.bool,
               'Object_Index': np.uint8, "Start_Time": np.datetime64, 'Detected': np.bool,
               'Max_Intersected_CNT': np.uint8, 'IOU': np.float, 'X': np.uint8, 'Y': np.uint8,
               'W': np.uint8, 'H': np.uint8, 'Frame_Int': np.float, 'GT_Int': np.float})


columns = ['Frame_Number', 'FLIR0_Video1', 'Object_Index', 'Start_Time', 'True_False_Plus', 'True_False_Minus', 'Intersected_GTs', 'IOUs',
           'X', 'Y', 'W', 'H', 'Frame_Int', 'CNT_Int', 'Canny']
video_pred_table = pd.DataFrame(columns=columns)
video_pred_table = video_pred_table.astype({"Frame_Number": str, 'FLIR0_Video1': np.bool,
               'Object_Index': np.uint8, "Start_Time": np.datetime64, 'True_False_Plus': np.bool, 'True_False_Minus': np.bool,
               'Intersected_GTs': object, 'IOUs': object, 'X': np.uint8, 'Y': np.uint8,
               'W': np.uint8, 'H': np.uint8, 'Frame_Int': np.uint8, 'CNT_Int': np.float, 'Canny': np.float})


for im_path in list(video_gt.keys()):
    gts = video_gt[im_path]
    frame = cv2.imread(im_path, 0)
    frame_int = frame.mean()
    
    if im_path in list(video_preds.keys()):
        preds = video_preds[im_path]
    else:
        preds = []
        
    table = main_metric(gts, preds, frame.shape)
    
    maximums = []
    for i, gt in enumerate(gts):
        max_Iou = 0
        x,y,w,h = gt
        gt_int = frame[y:y+h,x:x+w].mean()
        if table.shape[0] * table.shape[1] != 0 :
            
            
            IoUs = table[i,:,0]
            IoOs = table[i,:,1]
            
            max_IoU = IoUs.max()
            max_IoO = IoOs.max()
            
            max_cnt = None
            
            is_true = check_true(IoUs, IoOs, metric_type)
            
        else:
            max_IoU = 0
            max_IoO = 0
            max_cnt = None
            is_true = False
            
        maximums.append(round(max_IoU, 3))

        video_gt_table = video_gt_table.append(pd.Series(
                        ['Video_'+im_path[-9:-5], True, i, pd.Timestamp('now'), is_true, max_cnt, max_IoU, 
                             x, y, w, h, frame_int, gt_int]
                            , index=video_gt_table.columns), ignore_index=True)
        
    for j, pred in enumerate(preds):
        x,y,w,h = pred
        pred_int = frame[y:y+h,x:x+w].mean()
        if table.shape[0] * table.shape[1] != 0 :
            IoUs = table[:,j,0]
            IoOs = table[:,j,1]
            
            max_IoU = IoUs.max()
            max_IoO = IoOs.max()
            is_true = check_true(IoUs, IoOs, metric_type)
            
            is_true_m = False
            
            for iou in IoUs:
                if round(iou,3) in maximums and is_true:
                    is_true_m = True
                
            intersected_gts = None
            IoUs = None
        else:
            max_IoU = 0
            max_IoO = None
            intersected_gts = None
            IoUs = None
            is_true = False
            is_true_m = False
        video_pred_table = video_pred_table.append(pd.Series(
                ['Video_'+im_path[-9:-5], True, j, pd.Timestamp('now'), is_true, is_true_m, 
                 intersected_gts, IoUs, x, y, w, h, frame_int, pred_int, 0]
                , index=video_pred_table.columns), ignore_index=True)


Wall time: 26.3 s


In [44]:
display(video_gt_table)
display(video_pred_table)

,Frame_Number,FLIR0_Video1,Object_Index,Start_Time,Detected,Max_Intersected_CNT,IOU,X,Y,W,H,Frame_Int,GT_Int
0,Video_1500,True,0,2020-07-21 22:16:16.633503,True,None,0.864552,509,148,82,173,93.745501,111.821655
1,Video_1500,True,1,2020-07-21 22:16:16.640500,True,None,0.561813,564,81,22,36,93.745501,137.731061
2,Video_1500,True,2,2020-07-21 22:16:16.648499,True,None,0.561813,551,80,17,37,93.745501,175.394277
3,Video_1501,True,0,2020-07-21 22:16:16.691505,True,None,0.864552,509,148,82,173,93.719501,111.821585
4,Video_1501,True,1,2020-07-21 22:16:16.700507,True,None,0.753392,564,81,22,36,93.719501,136.877525
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1449,Video_8228,True,0,2020-07-21 22:16:42.699356,False,None,0.000000,857,129,57,113,95.827541,94.783574
1450,Video_8229,True,0,2020-07-21 22:16:42.718357,False,None,0.000000,871,132,51,113,96.892685,94.633524
1451,Video_8230,True,0,2020-07-21 22:16:42.744359,False,None,0.000000,878,134,46,105,97.260321,92.808489
1452,Video_8231,True,0,2020-07-21 22:16:42.760358,False,None,0.000000,892,134,31,106,97.662397,103.525867


,Frame_Number,FLIR0_Video1,Object_Index,Start_Time,True_False_Plus,True_False_Minus,Intersected_GTs,IOUs,X,Y,W,H,Frame_Int,CNT_Int,Canny
0,Video_1500,True,0,2020-07-21 22:16:16.655500,True,True,None,None,549,66,41,55,93.745501,139.436807,0.0
1,Video_1500,True,1,2020-07-21 22:16:16.667500,True,True,None,None,508,162,79,159,93.745501,120.159860,0.0
2,Video_1501,True,0,2020-07-21 22:16:16.728522,True,True,None,None,508,162,79,159,93.719501,120.158905,0.0
3,Video_1501,True,1,2020-07-21 22:16:16.741512,True,True,None,None,548,78,39,43,93.719501,140.731067,0.0
4,Video_1502,True,0,2020-07-21 22:16:16.827517,True,True,None,None,508,162,79,159,93.770488,120.165592,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
956,Video_8226,True,1,2020-07-21 22:16:42.638350,False,False,None,None,659,368,125,91,96.166654,120.642989,0.0
957,Video_8227,True,0,2020-07-21 22:16:42.679354,False,False,None,None,686,368,121,115,96.368538,117.262379,0.0
958,Video_8229,True,0,2020-07-21 22:16:42.728357,False,False,None,None,725,363,84,92,96.892685,122.669902,0.0
959,Video_8247,True,0,2020-07-21 22:16:42.910350,False,False,None,None,642,388,116,73,100.720578,116.560109,0.0


In [45]:
video_pred_table[video_pred_table.Frame_Number == 'Video_1740']

,Frame_Number,FLIR0_Video1,Object_Index,Start_Time,True_False_Plus,True_False_Minus,Intersected_GTs,IOUs,X,Y,W,H,Frame_Int,CNT_Int,Canny
536,Video_1740,True,0,2020-07-21 22:16:32.277853,True,True,None,None,440,155,159,375,85.405682,94.698365,0.0
537,Video_1740,True,1,2020-07-21 22:16:32.284854,True,True,None,None,676,97,40,81,85.405682,133.882099,0.0
538,Video_1740,True,2,2020-07-21 22:16:32.291860,True,True,None,None,710,199,98,204,85.405682,109.413065,0.0


In [46]:
print("Ошибка 2 рода: ", 1 - video_gt_table['Detected'].sum()/video_gt_table['Detected'].count(), 
     "Ошибка 1 рода minus: ", 1 - video_pred_table['True_False_Minus'].sum()/video_pred_table['True_False_Minus'].count(),
     "Ошибка 1 рода plus: ", 1 - video_pred_table['True_False_Plus'].sum()/video_pred_table['True_False_Plus'].count(),
     "Количество ложноположительных контуров plus: ", video_pred_table['True_False_Plus'].count() - video_pred_table['True_False_Plus'].sum(),
     "Количество ложноположительных контуров minus: ", video_pred_table['True_False_Minus'].count() - video_pred_table['True_False_Minus'].sum())

Ошибка 2 рода:  0.16781292984869323 Ошибка 1 рода minus:  0.0915712799167534 Ошибка 1 рода plus:  0.08740894901144636 Количество ложноположительных контуров plus:  84 Количество ложноположительных контуров minus:  88


In [39]:
print("Ошибка 2 рода: ", 1 - video_gt_table['Detected'].sum()/video_gt_table['Detected'].count(), 
     "Ошибка 1 рода minus: ", 1 - video_pred_table['True_False_Minus'].sum()/video_pred_table['True_False_Minus'].count(),
     "Ошибка 1 рода plus: ", 1 - video_pred_table['True_False_Plus'].sum()/video_pred_table['True_False_Plus'].count(),
     "Количество ложноположительных контуров plus: ", video_pred_table['True_False_Plus'].count() - video_pred_table['True_False_Plus'].sum(),
     "Количество ложноположительных контуров minus: ", video_pred_table['True_False_Minus'].count() - video_pred_table['True_False_Minus'].sum())

Ошибка 2 рода:  0.15887207702888584 Ошибка 1 рода minus:  0.1165829145728643 Ошибка 1 рода plus:  0.11256281407035174 Количество ложноположительных контуров plus:  112 Количество ложноположительных контуров minus:  116


In [50]:
print("Ошибка 2 рода: ", 1 - video_gt_table['Detected'].sum()/video_gt_table['Detected'].count(), 
     "Ошибка 1 рода minus: ", 1 - video_pred_table['True_False_Minus'].sum()/video_pred_table['True_False_Minus'].count(),
     "Ошибка 1 рода plus: ", 1 - video_pred_table['True_False_Plus'].sum()/video_pred_table['True_False_Plus'].count(),
     "Количество ложноположительных контуров plus: ", video_pred_table['True_False_Plus'].count() - video_pred_table['True_False_Plus'].sum(),
     "Количество ложноположительных контуров minus: ", video_pred_table['True_False_Minus'].count() - video_pred_table['True_False_Minus'].sum())

Ошибка 2 рода:  0.8129298486932599 Ошибка 1 рода minus:  0.0239043824701195 Ошибка 1 рода plus:  0.0239043824701195 Количество ложноположительных контуров plus:  6 Количество ложноположительных контуров minus:  6


In [19]:
print("Ошибка 2 рода: ", 1 - video_gt_table['Detected'].sum()/video_gt_table['Detected'].count(), 
     "Ошибка 1 рода minus: ", 1 - video_pred_table['True_False_Minus'].sum()/video_pred_table['True_False_Minus'].count(),
     "Ошибка 1 рода plus: ", 1 - video_pred_table['True_False_Plus'].sum()/video_pred_table['True_False_Plus'].count(),
     "Количество ложноположительных контуров plus: ", video_pred_table['True_False_Plus'].count() - video_pred_table['True_False_Plus'].sum(),
     "Количество ложноположительных контуров minus: ", video_pred_table['True_False_Minus'].count() - video_pred_table['True_False_Minus'].sum())

Ошибка 2 рода:  0.23590096286107287 Ошибка 1 рода minus:  0.5978147762747139 Ошибка 1 рода plus:  0.5978147762747139 Количество ложноположительных контуров plus:  1149 Количество ложноположительных контуров minus:  1149


In [93]:
print("Ошибка 2 рода: ", 1 - video_gt_table['Detected'].sum()/video_gt_table['Detected'].count(), 
     "Ошибка 1 рода minus: ", 1 - video_pred_table['True_False_Minus'].sum()/video_pred_table['True_False_Minus'].count(),
     "Ошибка 1 рода plus: ", 1 - video_pred_table['True_False_Plus'].sum()/video_pred_table['True_False_Plus'].count(),
     "Количество ложноположительных контуров plus: ", video_pred_table['True_False_Plus'].count() - video_pred_table['True_False_Plus'].sum(),
     "Количество ложноположительных контуров minus: ", video_pred_table['True_False_Minus'].count() - video_pred_table['True_False_Minus'].sum())

Ошибка 2 рода:  0.1911966987620357 Ошибка 1 рода minus:  0.8311478716965907 Ошибка 1 рода plus:  0.8311478716965907 Количество ложноположительных контуров plus:  4120 Количество ложноположительных контуров minus:  4120


In [56]:
print("Ошибка 2 рода: ", 1 - video_gt_table['Detected'].sum()/video_gt_table['Detected'].count(), 
     "Ошибка 1 рода minus: ", 1 - video_pred_table['True_False_Minus'].sum()/video_pred_table['True_False_Minus'].count(),
     "Ошибка 1 рода plus: ", 1 - video_pred_table['True_False_Plus'].sum()/video_pred_table['True_False_Plus'].count(),
     "Количество ложноположительных контуров plus: ", video_pred_table['True_False_Plus'].count() - video_pred_table['True_False_Plus'].sum(),
     "Количество ложноположительных контуров minus: ", video_pred_table['True_False_Minus'].count() - video_pred_table['True_False_Minus'].sum())

Ошибка 2 рода:  0.23383768913342506 Ошибка 1 рода minus:  0.8589420654911839 Ошибка 1 рода plus:  0.8589420654911839 Количество ложноположительных контуров plus:  4774 Количество ложноположительных контуров minus:  4774


In [18]:
%%time

columns = ['Frame_Number', 'FLIR0_Video1', 'Object_Index', 'Start_Time', 'Detected', 'Max_Intersected_CNT', 'IOU', 
          'X', 'Y', 'W', 'H', 'Frame_Int', 'GT_Int']
gt_table = pd.DataFrame(columns=columns)
gt_table = gt_table.astype({"Frame_Number": str, 'FLIR0_Video1': np.bool,
               'Object_Index': np.uint8, "Start_Time": np.datetime64, 'Detected': np.bool,
               'Max_Intersected_CNT': np.uint8, 'IOU': np.float, 'X': np.uint8, 'Y': np.uint8,
               'W': np.uint8, 'H': np.uint8, 'Frame_Int': np.float, 'GT_Int': np.float})


columns = ['Frame_Number', 'FLIR0_Video1', 'Object_Index', 'Start_Time', 'True_False_Plus', 'True_False_Minus', 'Intersected_GTs', 'IOUs',
           'X', 'Y', 'W', 'H', 'Frame_Int', 'CNT_Int', 'Canny']
pred_table = pd.DataFrame(columns=columns)
pred_table = pred_table.astype({"Frame_Number": str, 'FLIR0_Video1': np.bool,
               'Object_Index': np.uint8, "Start_Time": np.datetime64, 'True_False_Plus': np.bool, 'True_False_Minus': np.bool,
               'Intersected_GTs': object, 'IOUs': object, 'X': np.uint8, 'Y': np.uint8,
               'W': np.uint8, 'H': np.uint8, 'Frame_Int': np.uint8, 'CNT_Int': np.float, 'Canny': np.float})


for im_path in list(FLIR_val_gt.keys()):
    gts = FLIR_val_gt[im_path]
    frame = cv2.imread(im_path, 0)
    frame_int = frame.mean()
    
    if im_path in list(FLIR_val_preds.keys()):
        preds = FLIR_val_preds[im_path]
    else:
        preds = []
    
    table = main_metric(gts, preds, frame.shape)

    maximums = []
    for i, gt in enumerate(gts):
        x,y,w,h = gt
        gt_int = frame[y:y+h,x:x+w].mean()
        if table.shape[0] * table.shape[1] != 0 :
            IoUs = table[i,:,0]
            IoOs = table[i,:,1]
            
            max_IoU = IoUs.max()
            max_IoO = IoOs.max()
            
            max_cnt = None
            
            is_true = check_true(IoUs, IoOs, metric_type)
            
        else:
            max_IoU = 0
            max_IoO = 0
            max_cnt = None
            is_true = False
        
        maximums.append(round(max_IoU, 3))
        
        gt_table = gt_table.append(pd.Series(
                        ['FLIR_'+im_path[-9:-5], False, i, pd.Timestamp('now'), is_true, max_cnt, max_IoU, 
                             x, y, w, h, frame_int, gt_int]
                            , index=gt_table.columns), ignore_index=True)
        
    for j, pred in enumerate(preds):
        x,y,w,h = pred
        pred_int = frame[y:y+h,x:x+w].mean()
        if table.shape[0] * table.shape[1] != 0 :
            IoUs = table[:,j,0]
            IoOs = table[:,j,1]
            
            max_IoU = IoUs.max()
            max_IoO = IoOs.max()
            is_true = check_true(IoUs, IoOs, metric_type)
            
            is_true_m = False
            
            for iou in IoUs:
                if round(iou,3) in maximums and is_true:
                    is_true_m = True
                
            
            intersected_gts = None
            IoUs = None
        else:
            max_IoU = 0
            max_IoO = None
            intersected_gts = None
            IoUs = None
            is_true = False
            is_true_m = False
            
        pred_table = pred_table.append(pd.Series(
                ['FLIR_'+im_path[-9:-5], False, j, pd.Timestamp('now'), is_true, is_true_m,
                 intersected_gts, IoUs, x, y, w, h, frame_int, pred_int, 0]
                , index=pred_table.columns), ignore_index=True)

Wall time: 45.6 s


In [19]:
display(gt_table)
display(pred_table)

,Frame_Number,FLIR0_Video1,Object_Index,Start_Time,Detected,Max_Intersected_CNT,IOU,X,Y,W,H,Frame_Int,GT_Int
0,FLIR_8863,False,0,2020-07-23 19:18:11.800848,False,None,0.024300,580,206,34,75,140.909201,209.165098
1,FLIR_8863,False,1,2020-07-23 19:18:11.816855,False,None,0.024300,609,207,30,116,140.909201,215.918103
2,FLIR_8864,False,0,2020-07-23 19:18:11.904871,True,None,0.609890,369,193,74,126,127.433084,141.401652
3,FLIR_8864,False,1,2020-07-23 19:18:11.912877,True,None,0.239868,488,192,49,134,127.433084,172.186567
4,FLIR_8864,False,2,2020-07-23 19:18:11.920874,True,None,0.239868,544,200,32,104,127.433084,202.148438
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2138,FLIR_9651,False,0,2020-07-23 19:18:56.686325,False,None,0.000000,114,188,40,137,136.708417,212.900000
2139,FLIR_9651,False,1,2020-07-23 19:18:56.704877,False,None,0.000000,152,167,60,187,136.708417,211.818806
2140,FLIR_9652,False,0,2020-07-23 19:18:56.728833,True,None,0.372348,410,215,24,56,131.505838,210.787202
2141,FLIR_9652,False,1,2020-07-23 19:18:56.739105,True,None,0.372348,433,216,13,55,131.505838,215.061538


,Frame_Number,FLIR0_Video1,Object_Index,Start_Time,True_False_Plus,True_False_Minus,Intersected_GTs,IOUs,X,Y,W,H,Frame_Int,CNT_Int,Canny
0,FLIR_8863,False,0,2020-07-23 19:18:11.840854,False,False,None,None,391,223,27,31,140.909201,119.637993,0.0
1,FLIR_8863,False,1,2020-07-23 19:18:11.856893,False,False,None,None,0,150,640,362,140.909201,171.347354,0.0
2,FLIR_8863,False,2,2020-07-23 19:18:11.872870,False,False,None,None,0,91,387,198,140.909201,153.073839,0.0
3,FLIR_8864,False,0,2020-07-23 19:18:11.936884,False,False,None,None,120,231,96,50,127.433084,142.777708,0.0
4,FLIR_8864,False,1,2020-07-23 19:18:11.948282,True,True,None,None,360,185,104,147,127.433084,132.393446,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2045,FLIR_9666,False,2,2020-07-23 19:18:57.264630,False,False,None,None,326,139,21,32,128.855215,87.483631,0.0
2046,FLIR_9666,False,3,2020-07-23 19:18:57.272636,False,False,None,None,584,209,16,47,128.855215,159.573138,0.0
2047,FLIR_9667,False,0,2020-07-23 19:18:57.288644,False,False,None,None,229,140,24,38,141.592252,111.763158,0.0
2048,FLIR_9667,False,1,2020-07-23 19:18:57.296648,False,False,None,None,0,86,360,223,141.592252,130.904260,0.0


In [24]:
print("Ошибка 2 рода: ", 1 - gt_table['Detected'].sum()/gt_table['Detected'].count(), 
     "Ошибка 1 рода minus: ", 1 - pred_table['True_False_Minus'].sum()/pred_table['True_False_Minus'].count(),
     "Ошибка 1 рода plus: ", 1 - pred_table['True_False_Plus'].sum()/pred_table['True_False_Plus'].count(),
     "Количество ложноположительных контуров plus: ", pred_table['True_False_Plus'].count() - pred_table['True_False_Plus'].sum(),
     "Количество ложноположительных контуров minus: ", pred_table['True_False_Minus'].count() - pred_table['True_False_Minus'].sum())

Ошибка 2 рода:  0.49883341110592627 Ошибка 1 рода minus:  0.4708121827411168 Ошибка 1 рода plus:  0.4708121827411168 Количество ложноположительных контуров plus:  371 Количество ложноположительных контуров minus:  371
